In [ ]:
import numpy as np
import h5py
import math
import time
import datetime
import matplotlib.pyplot as plt
import keras.backend as K
from keras.models import model_from_json
import tensorflow as tf
from scipy.stats import norm, expon, chi2
from scipy.stats import chisquare
import os



import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.constraints import Constraint
from tensorflow.keras import metrics, losses, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Layer
from tensorflow import Variable

In [ ]:
columns = {
    'pt1' : 0,
    'pt2': 1,
    'eta1': 2,
    'eta2': 3, 
    'delta_phi':4,
    'mass' : 5
}

def get_data(seed, NNmodel):
    #....
    # note: 
    # D and R are taken before standardization; keep the mass variable (6 columns)
    # output_D and output_R are the NN output (on the standardized D[:, :-1] and R[:, :-1]); 
    # weights_D is a vector of ones, weights_R is a vector of N(R)/N_r
    return D, R, weights_D, weights_R, output_D, output_R 


def PlotPrediction(seed, NNmodel, bins_plot, key='mass', logx=False, output_path='', save=False):
    if not key in list(columns.keys()):
        print('Wrong key! choose between:')
        print(list(columns.keys()))
        return
    
    Data, Ref, wData, wRef, f_Data, f_Ref = get_data(seed, NNmodel)
    
    xRef  = Ref[:, columns[key]]
    xData = Data[:, olumns[key]]
    wNN   = np.exp(f_Ref)*weightsRef
    
    plt.subplots(1, 2, figsize=(20, 8))
    # NN reconstruction
    hNN   = plt.hist(xRef, bins=bins_plot, weights=wNN, alpha=0, )
    # Reference
    hRef = plt.hist(xRef, bins=bins_plot, weights=wRef, alpha=0, )
    # Data
    hData = plt.hist(xData, bins=bins_plot, weights=wData, ec='orange', lw=1, color='gold', label='Data')
    plt.subplot(1, 2, 1)
    x = 0.5*(hNN[1][1:]+hNN[1][:-1])
    plt.plot(x, hNN[0],  ls='--', color='blue', label='NN prediction' )
    plt.plot(x, hRef[0], ls='--', color='red',  label='Reference' )
    plt.legend(fontsize=14)
    plt.xlabel(key, fontsize=14)
    plt.ylabel('Counts', fontsize=14)
    plt.yscale('log')
    if logx:
        plt.xscale('log')
    
    plt.subplot(1, 2, 2)
    ratio_reco   = np.divide(hNN[0],hRef[0]+1e-8)
    ratio_data   = np.divide(hData[0],hRef[0]+1e-8)
    x            = 0.5*(bins_plot[1:]+bins_plot[:-1])
    
    plt.plot(x, ratio_data, marker='o', color='darkorange', linewidth=3, label='Data')
    plt.plot(x, ratio_reco, marker='o', color='blue',       linewidth=3, label='NN reconstruction')
    plt.legend(fontsize=14)
    plt.xlabel(key, fontsize=14)
    plt.ylabel('n(x|T)/n(x|R)', fontsize=14)
    if logx:
        plt.xscale('log')
        
    plt.show()
    if save:
        if output_path=='':
            print('output_path not defined. The figure will not be saved')
        else:
            plt.savefig(output_path+'/NNreconstruction_%i_%s.png'%(seed, key))
    plt.close()
    return